In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
from tqdm import tqdm
import random
import os
import matplotlib.pyplot as plt

# --- Configuration for Data Loading and SimpleNN ---
BATCH_SIZE = 64
NUM_EPOCHS_SIMPLE_NN = 3 # For initial training of Class 1/2 models and target models (if generated)
LEARNING_RATE_SIMPLE_NN = 0.001

# Define the classes for each subset (same as before)
CLASS1_LABELS = [0, 1, 2, 3]
CLASS2_LABELS = [2, 3, 4, 5]
TARGET_MODEL_LABELS = [0, 1, 2, 3, 4] # Labels for the "ideal updated Model 1"

# Evaluation labels for the final predicted model
EVAL_SET_A_LABELS = [0, 1, 2, 3, 4] # Reverted to focus on digits 0-4
EVAL_SET_B_LABELS = [5] # Still defined, but its loss component will not be part of meta-loss
ALL_EVAL_DIGITS = [0, 1, 2, 3, 4, 5]

# --- Configuration for MetaModel Training ---
# Increased parameters for more rigorous training
NUM_TARGET_MODELS = 50 # Number of "ideal updated Model 1" models to generate (if needed)
NUM_META_MODEL_TRAINING_PAIRS = 1000 # Increased training pairs (from 500)
META_MODEL_EPOCHS = 200 # Increased epochs (from 100)
META_MODEL_LEARNING_RATE = 0.0005
META_MODEL_BATCH_SIZE = 64

# New parameter for sampled internal evaluation - now using more batches
NUM_BATCHES_FOR_INTERNAL_EVAL = 20 # Increased sampled batches for custom loss calculation (from 5)

# Interval to print MetaModel training loss (in batches)
META_LOSS_PRINT_INTERVAL = 25

# Set device to GPU if available, otherwise CPU
# Prioritize MPS for Apple Silicon, then CUDA, then CPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
print(f"Using device: {DEVICE}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
elif torch.backends.mps.is_available(): # For Apple Silicon
    # MPS does not have a manual_seed_all equivalent for all operations.
    # Deterministic behavior for MPS is generally handled by setting seeds for CPU/GPU.
    # For full reproducibility, you might need to disable MPS for some operations,
    # or accept minor variations.
    pass


# --- Data Loading and Preprocessing (MNIST) ---
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) # MNIST mean and std
])

print("Loading MNIST dataset...")
full_train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
full_test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)
print("MNIST dataset loaded.")

# --- Dataset Subset Creation Function ---
def create_subset(dataset, labels_to_include):
    """Filters a dataset to include only samples with specified labels."""
    indices = []
    for i, (_, label) in enumerate(dataset):
        if label in labels_to_include:
            indices.append(i)
    return Subset(dataset, indices)

# Create datasets for SimpleNN training and evaluation
class1_train_dataset = create_subset(full_train_dataset, CLASS1_LABELS)
class2_train_dataset = create_subset(full_train_dataset, CLASS2_LABELS)
target_model_train_dataset = create_subset(full_train_dataset, TARGET_MODEL_LABELS)

# Evaluation datasets for the final predicted model
# These loaders will also be used *inside* the MetaModel's training loop for custom loss
eval_loader_A_internal = DataLoader(create_subset(full_test_dataset, EVAL_SET_A_LABELS), batch_size=INTERNAL_EVAL_BATCH_SIZE, shuffle=True) # Shuffle for sampling
eval_loader_B_internal = DataLoader(create_subset(full_test_dataset, EVAL_SET_B_LABELS), batch_size=INTERNAL_EVAL_BATCH_SIZE, shuffle=True) # Shuffle for sampling


# Create individual digit test datasets for detailed tracing
individual_digit_test_loaders = {}
for digit in ALL_EVAL_DIGITS:
    digit_dataset = create_subset(full_test_dataset, [digit])
    if len(digit_dataset) > 0:
        individual_digit_test_loaders[digit] = DataLoader(digit_dataset, batch_size=BATCH_SIZE, shuffle=False)

# --- SimpleNN Model Definition (ONE HIDDEN LAYER REMOVED) ---
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # Input layer: 28*28 = 784 pixels
        # Single Hidden layer: 8 neurons (reduced from 16)
        # Output layer: 10 neurons (for digits 0-9)
        self.fc1 = nn.Linear(28 * 28, 10) # Input to the single hidden layer, reduced to 8 neurons
        # self.relu1 = nn.ReLU()
        # self.fc3 = nn.Linear(16, 10) # Direct connection from hidden layer to output, input size matches fc1 output

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        # x = self.relu1(x)
        # x = self.fc3(x) # Direct connection
        return x

# --- Helper Functions for Flattening/Unflattening State Dicts ---
def flatten_state_dict(state_dict):
    """Flattens a state_dict into a single 1D tensor."""
    return torch.cat([param.view(-1) for param in state_dict.values()])

def unflatten_tensor_to_state_dict(flat_tensor, model_template_state_dict):
    """Unflattens a 1D tensor back into a state_dict, using a template."""
    new_state_dict = {}
    offset = 0
    for name, param in model_template_state_dict.items():
        num_elements = param.numel()
        # Ensure the tensor is on the correct device and has requires_grad if needed
        # For loading into a model that will be evaluated for loss, it needs to track gradients.
        new_state_dict[name] = flat_tensor[offset:offset + num_elements].view(param.shape).to(param.device)
        offset += num_elements
    return new_state_dict

# Get a template model to determine state_dict structure and size
template_model = SimpleNN() # Re-initialize to get the new, smaller architecture's state_dict
template_state_dict = template_model.state_dict()
FLATTENED_STATE_DICT_SIZE = flatten_state_dict(template_state_dict).numel()
print(f"Flattened SimpleNN state_dict size: {FLATTENED_STATE_DICT_SIZE}")

# --- MetaModel Definition (REDUCED CAPACITY) ---
class MetaModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(MetaModel, self).__init__()
        # Input: flattened state_dict of Model 1 + flattened state_dict of (Model 2 - Model 1)
        # Output: flattened state_dict of the *difference* to add to Model 1
        # Hidden layer sizes significantly reduced
        self.fc1 = nn.Linear(input_size, 512) # Reduced from 2048
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512, 256) # Reduced from 1024
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(256, 128) # Reduced from 512
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(128, output_size) # Output is the flattened state_dict

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        return x

# --- Training and Evaluation Functions (SimpleNN) ---
def train_simple_nn(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        for data, target in train_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

def evaluate_simple_nn_accuracy(model, data_loader):
    """Evaluates a SimpleNN model and returns accuracy (for final reporting)."""
    model.eval() # Set to eval mode, no gradients needed for this accuracy calculation
    correct = 0
    total = 0
    if len(data_loader.dataset) == 0:
        return float('nan')
    with torch.no_grad(): # No gradients needed for accuracy calculation
        for data, target in data_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    return 100 * correct / total

def calculate_classification_loss(model, data_loader, criterion, num_batches_to_sample=None):
    """
    Calculates the average classification loss for a SimpleNN model over a DataLoader.
    Can sample a fixed number of batches for lighter computation.
    This function is designed to be differentiable for the MetaModel's loss.
    """
    model.train() # Keep in train mode to ensure gradients are tracked
    total_loss = 0.0
    num_batches_processed = 0
    
    # Create an iterator for the DataLoader
    data_iter = iter(data_loader)

    for _ in range(num_batches_to_sample if num_batches_to_sample is not None else len(data_loader)):
        try:
            data, target = next(data_iter)
        except StopIteration:
            # Reached end of dataset, reset iterator for continuous sampling
            data_iter = iter(data_loader)
            data, target = next(data_iter) # Get next batch from the beginning

        data, target = data.to(DEVICE), target.to(DEVICE)
        output = model(data)
        loss = criterion(output, target)
        total_loss += loss # Accumulate loss directly (not .item())
        num_batches_processed += 1
    
    return total_loss / num_batches_processed if num_batches_processed > 0 else torch.tensor(0.0, device=DEVICE)


# --- Main Logic ---
# Consistent SAVE_DIR with the first script
SAVE_DIR = './trained_models_smallest'
PLOTS_DIR = './meta_plots'
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)

# Define paths for saved models (corrected filenames)
class1_save_path = os.path.join(SAVE_DIR, 'class1_models_weights.pt')
class2_save_path = os.path.join(SAVE_DIR, 'class2_models_weights.pt')
target_models_save_path = os.path.join(SAVE_DIR, 'target_updated_models.pt') # Still used for generating target models if not found

# 1. Load initial Class 1 and Class 2 models (STRICTLY LOAD)
class1_models = []
class2_models = []

print("Attempting to load pre-generated Class 1 and Class 2 models...")
try:
    if os.path.exists(class1_save_path):
        class1_models = torch.load(class1_save_path, map_location=DEVICE, weights_only=True)
        print(f"Loaded {len(class1_models)} Class 1 models.")
    else:
        raise FileNotFoundError(f"Class 1 model weights file not found at {class1_save_path}. Please run the 'MNIST Subset Neural Network Experiment' Canvas first to generate these models.")

    if os.path.exists(class2_save_path):
        class2_models = torch.load(class2_save_path, map_location=DEVICE, weights_only=True)
        print(f"Loaded {len(class2_models)} Class 2 models.")
    else:
        raise FileNotFoundError(f"Class 2 model weights file not found at {class2_save_path}. Please run the 'MNIST Subset Neural Network Experiment' Canvas first to generate these models.")

except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Exiting. Please ensure the initial Class 1 and Class 2 models are generated and saved in the correct directory.")
    exit() # Exit if essential files are missing
except Exception as e:
    print(f"An unexpected error occurred while loading initial models: {e}")
    exit()

# 2. Load "Target Updated Model 1" weights (models trained on 0,1,2,3,4)
target_updated_models = []
print("Attempting to load pre-generated Target Updated Model 1 weights...")
try:
    if os.path.exists(target_models_save_path):
        target_updated_models = torch.load(target_models_save_path, map_location=DEVICE, weights_only=True)
        print(f"Loaded {len(target_updated_models)} Target Updated Model 1 models.")
    else:
        # This part remains a fallback to generate target models if they don't exist,
        # as they are the "ground truth" for what the meta-model should aim to produce.
        print(f"Target updated model weights file not found at {target_models_save_path}. Generating them now...")
        for _ in tqdm(range(NUM_TARGET_MODELS), desc="Generating Target Updated Models"):
            model = SimpleNN().to(DEVICE)
            train_simple_nn(model, DataLoader(target_model_train_dataset, BATCH_SIZE, True), nn.CrossEntropyLoss(), optim.Adam(model.parameters(), LEARNING_RATE_SIMPLE_NN), NUM_EPOCHS_SIMPLE_NN * 2) # Train a bit more for good performance
            target_updated_models.append(model.state_dict())
        torch.save(target_updated_models, target_models_save_path)
        print("Target Updated Model 1 models generated and saved.")

except Exception as e:
    print(f"An unexpected error occurred while loading/generating target models: {e}")
    exit()

# Ensure we have enough models after loading/generating
if not class1_models or not class2_models or not target_updated_models:
    raise RuntimeError("Critical: Not enough models available after loading/generation. Something went wrong.")


# 3. Prepare training data for the MetaModel (inputs: original Model1, Model2-Model1 diff)
meta_train_inputs = []

print(f"Preparing {NUM_META_MODEL_TRAINING_PAIRS} training inputs for the MetaModel...")
for _ in tqdm(range(NUM_META_MODEL_TRAINING_PAIRS), desc="Preparing MetaModel data"):
    # Randomly select one Class 1 model and one Class 2 model
    original_class1_sd = random.choice(class1_models)
    class2_sd = random.choice(class2_models)

    # Flatten inputs
    flat_original_class1 = flatten_state_dict(original_class1_sd).to(DEVICE)
    flat_class2 = flatten_state_dict(class2_sd).to(DEVICE)
    
    # Calculate the difference in weights
    diff_m2_m1_flat = flat_class2 - flat_original_class1

    # Concatenate original Model 1 weights and the difference as MetaModel input
    meta_input = torch.cat((flat_original_class1, diff_m2_m1_flat))

    meta_train_inputs.append(meta_input)

# Convert lists to tensors for DataLoader
meta_train_inputs_tensor = torch.stack(meta_train_inputs)

# Create a dummy tensor for targets in TensorDataset, as targets are computed dynamically
dummy_targets = torch.zeros(meta_train_inputs_tensor.shape[0], 1, device=DEVICE)
meta_dataset = torch.utils.data.TensorDataset(meta_train_inputs_tensor, dummy_targets)
meta_dataloader = DataLoader(meta_dataset, batch_size=META_MODEL_BATCH_SIZE, shuffle=True)


# 4. Define and Train the MetaModel
meta_model_input_size = FLATTENED_STATE_DICT_SIZE * 2 # Original Model 1 + (Model 2 - Model 1)
meta_model_output_size = FLATTENED_STATE_DICT_SIZE # Predicted difference to add to Model 1

meta_model = MetaModel(meta_model_input_size, meta_model_output_size).to(DEVICE)
meta_criterion_ce = nn.CrossEntropyLoss() # Used for internal SimpleNN evaluation
meta_optimizer = optim.Adam(meta_model.parameters(), lr=META_MODEL_LEARNING_RATE)

print(f"\n--- Training the MetaModel with Custom Loss ---")
meta_train_losses = []
for epoch in tqdm(range(META_MODEL_EPOCHS), desc="Training MetaModel"):
    meta_model.train() # Set MetaModel to train mode
    epoch_loss_sum = 0.0 # To track loss for printing
    num_batches_in_epoch = 0

    for batch_idx, (batch_inputs, _) in enumerate(meta_dataloader): # Dummy targets are ignored
        # batch_inputs contains [original_model1_flat, diff_m2_m1_flat]
        original_model1_flat_batch = batch_inputs[:, :FLATTENED_STATE_DICT_SIZE]
        # diff_m2_m1_flat_batch = batch_inputs[:, FLATTENED_STATE_DICT_SIZE:] # Not directly used here but conceptually part of input

        batch_inputs = batch_inputs.to(DEVICE)
        meta_optimizer.zero_grad()

        # Get predicted flattened difference from MetaModel
        predicted_diff_for_m1_flat_batch = meta_model(batch_inputs) # This tensor has requires_grad=True

        batch_meta_loss = 0.0
        current_batch_loss_0_4 = 0.0 # Now representing loss on 0,1,2,3,4
        # current_batch_loss_5 is no longer used in the meta_loss calculation

        # Iterate over each predicted model in the batch
        for i in range(predicted_diff_for_m1_flat_batch.shape[0]):
            single_predicted_diff_for_m1_flat = predicted_diff_for_m1_flat_batch[i]
            single_original_model1_flat = original_model1_flat_batch[i] # Get the corresponding original Model 1

            # Calculate the predicted updated Model 1 weights
            predicted_updated_flat_m1 = single_original_model1_flat + single_predicted_diff_for_m1_flat
            
            # Unflatten predicted weights to form a SimpleNN's state_dict
            # Ensure these tensors retain their gradient tracking
            predicted_sd = unflatten_tensor_to_state_dict(predicted_updated_flat_m1, template_state_dict)
            
            # Instantiate a temporary SimpleNN and load the predicted state_dict
            temp_simple_nn = SimpleNN().to(DEVICE)
            temp_simple_nn.load_state_dict(predicted_sd)

            # Calculate loss components using the temporary SimpleNN on MNIST data
            # Sample a fixed number of batches for lighter computation
            loss_0_4 = calculate_classification_loss(temp_simple_nn, eval_loader_A_internal, meta_criterion_ce, num_batches_to_sample=NUM_BATCHES_FOR_INTERNAL_EVAL)
            # loss_5 is calculated but not used in the meta-loss for optimization
            # loss_5 = calculate_classification_loss(temp_simple_nn, eval_loader_B_internal, meta_criterion_ce, num_batches_to_sample=NUM_BATCHES_FOR_INTERNAL_EVAL)

            # Custom loss: Minimize loss on 0-4 ONLY
            batch_meta_loss += loss_0_4 # Only minimize loss on 0-4
            current_batch_loss_0_4 += loss_0_4.item()

        meta_loss = batch_meta_loss / predicted_diff_for_m1_flat_batch.shape[0] # Average over batch
        avg_loss_0_4 = current_batch_loss_0_4 / predicted_diff_for_m1_flat_batch.shape[0]
        # avg_loss_5 is not relevant for the meta-loss anymore, but can be calculated for monitoring if desired

        meta_loss.backward()
        meta_optimizer.step()
        meta_train_losses.append(meta_loss.item())
        epoch_loss_sum += meta_loss.item()
        num_batches_in_epoch += 1

        if (batch_idx + 1) % META_LOSS_PRINT_INTERVAL == 0:
            # Only print L_0-4 as it's the only component of the meta-loss
            print(f"    Epoch {epoch+1}/{META_MODEL_EPOCHS}, Batch {batch_idx+1}/{len(meta_dataloader)} - Avg Meta Loss: {epoch_loss_sum / num_batches_in_epoch:.4f} (L_0-4: {avg_loss_0_4:.4f})")
            epoch_loss_sum = 0.0 # Reset for next interval
            num_batches_in_epoch = 0 # Reset for next interval

    # Print final loss for the epoch if not already printed by interval
    if num_batches_in_epoch > 0:
        print(f"    Epoch {epoch+1}/{META_MODEL_EPOCHS}, Final Batch - Avg Meta Loss: {epoch_loss_sum / num_batches_in_epoch:.4f} (L_0-4: {avg_loss_0_4:.4f})")


print("MetaModel training complete.")

# Save the trained MetaModel
meta_model_save_path = os.path.join(SAVE_DIR, 'meta_model_custom_loss.pt')
torch.save(meta_model.state_dict(), meta_model_save_path)
print(f"MetaModel saved to {meta_model_save_path}")

# --- Plotting MetaModel Training Loss ---
plt.figure(figsize=(10, 6))
plt.plot(meta_train_losses)
plt.xlabel('MetaModel Training Step (Batch)')
plt.ylabel('Custom Loss (Loss_0-4)') # Updated label
plt.title('MetaModel Training Loss Over Steps (Custom Loss)')
plt.grid(True)
plot_save_path = os.path.join(PLOTS_DIR, 'meta_model_training_custom_loss.png')
plt.savefig(plot_save_path)
plt.close()


# 5. Evaluate the MetaModel's Output (by evaluating the predicted SimpleNN)
print("\n--- Evaluating the MetaModel's Output ---")

# Pick a random pair of initial Class 1 and Class 2 models for testing
test_original_class1_sd = random.choice(class1_models)
test_class2_sd = random.choice(class2_models)

# Flatten and concatenate for MetaModel input
flat_test_original_class1 = flatten_state_dict(test_original_class1_sd).to(DEVICE)
flat_test_class2 = flatten_state_dict(test_class2_sd).to(DEVICE)
diff_test_m2_m1_flat = flat_test_class2 - flat_test_original_class1
meta_test_input = torch.cat((flat_test_original_class1, diff_test_m2_m1_flat)).unsqueeze(0) # Add batch dimension

# Get predicted difference from MetaModel
meta_model.eval() # Set MetaModel to eval mode for final prediction
with torch.no_grad(): # No gradients needed for final prediction
    predicted_diff_for_m1_flat = meta_model(meta_test_input).squeeze(0) # Remove batch dimension

# Calculate the predicted updated Model 1 weights
predicted_updated_flat_m1 = flat_test_original_class1 + predicted_diff_for_m1_flat

# Unflatten predicted weights back into a state_dict
predicted_model_state_dict = unflatten_tensor_to_state_dict(predicted_updated_flat_m1.cpu(), template_state_dict)

# Instantiate a SimpleNN with the predicted weights
predicted_simple_nn = SimpleNN().to(DEVICE)
predicted_simple_nn.load_state_dict(predicted_model_state_dict)

# Evaluate the predicted SimpleNN on MNIST data (using accuracy)
print("\n--- Performance of the Model Predicted by MetaModel ---")
pred_acc_A = evaluate_simple_nn_accuracy(predicted_simple_nn, eval_loader_A_internal) # Use internal loaders for consistency
pred_acc_B = evaluate_simple_nn_accuracy(predicted_simple_nn, eval_loader_B_internal)

print(f"  Predicted Model Accuracy on {EVAL_SET_A_LABELS} (Accuracy A): {pred_acc_A:.2f}%")
print(f"  Predicted Model Accuracy on {EVAL_SET_B_LABELS} (Accuracy B): {pred_acc_B:.2f}%") # Still show for monitoring
print(f"  Predicted Model Custom Metric (A - B): {pred_acc_A - pred_acc_B:.2f}")

# Optional: Evaluate the original Class 1 model's performance for comparison
original_simple_nn = SimpleNN().to(DEVICE)
original_simple_nn.load_state_dict(test_original_class1_sd)
orig_acc_A = evaluate_simple_nn_accuracy(original_simple_nn, eval_loader_A_internal)
orig_acc_B = evaluate_simple_nn_accuracy(original_simple_nn, eval_loader_B_internal)
print(f"  Original Model Accuracy on {EVAL_SET_A_LABELS} (Accuracy A): {orig_acc_A:.2f}%")
print(f"  Original Model Accuracy on {EVAL_SET_B_LABELS} (Accuracy B): {orig_acc_B:.2f}%")
print(f"  Original Model Custom Metric (A - B): {orig_acc_A - orig_acc_B:.2f}")

print("\nMeta-Learning Experiment complete!")
print("Check the 'meta_plots' directory for visualizations of MetaModel training and output performance.")


Using device: mps
Loading MNIST dataset...
MNIST dataset loaded.
Flattened SimpleNN state_dict size: 7850
Attempting to load pre-generated Class 1 and Class 2 models...
Loaded 1000 Class 1 models.
Loaded 1000 Class 2 models.
Attempting to load pre-generated Target Updated Model 1 weights...
Loaded 50 Target Updated Model 1 models.
Preparing 1000 training inputs for the MetaModel...


Preparing MetaModel data: 100%|██████████| 1000/1000 [00:00<00:00, 11463.42it/s]



--- Training the MetaModel with Custom Loss ---


Training MetaModel:   0%|          | 1/200 [01:46<5:54:50, 106.99s/it]

    Epoch 1/200, Final Batch - Avg Meta Loss: 1.9395 (L_0-4: 1.9684)


...

Training MetaModel: 100%|█████████▉| 199/200 [6:37:20<01:46, 106.26s/it]

    Epoch 199/200, Final Batch - Avg Meta Loss: 1.9347 (L_0-4: 1.9267)


Training MetaModel: 100%|██████████| 200/200 [6:39:07<00:00, 119.74s/it]

    Epoch 200/200, Final Batch - Avg Meta Loss: 1.9364 (L_0-4: 1.9122)
MetaModel training complete.
MetaModel saved to ./trained_models_smallest/meta_model_custom_loss.pt

--- Evaluating the MetaModel's Output ---



--- Performance of the Model Predicted by MetaModel ---
  Predicted Model Accuracy on [0, 1, 2, 3, 4] (Accuracy A): 77.00%
  Predicted Model Accuracy on [5] (Accuracy B): 0.00%
  Predicted Model Custom Metric (A - B): 77.00
  Original Model Accuracy on [0, 1, 2, 3, 4] (Accuracy A): 78.81%
  Original Model Accuracy on [5] (Accuracy B): 0.00%
  Original Model Custom Metric (A - B): 78.81

Meta-Learning Experiment complete!
Check the 'meta_plots' directory for visualizations of MetaModel training and output performance.


In [ ]:
Using device: mps
Loading MNIST dataset...
MNIST dataset loaded.
Flattened SimpleNN state_dict size: 12730
Attempting to load pre-generated Class 1 and Class 2 models...
Loaded 20 Class 1 models.
Loaded 20 Class 2 models.
Attempting to load pre-generated Target Updated Model 1 weights...
Loaded 10 Target Updated Model 1 models.
Preparing 1000 training inputs for the MetaModel...
Preparing MetaModel data: 100%|██████████| 1000/1000 [00:00<00:00, 9507.53it/s]

--- Training the MetaModel with Custom Loss ---
Training MetaModel:   0%|          | 0/200 [00:00<?, ?it/s]    Epoch 1/200, Batch 25/63 - Avg Meta Loss: 1.8360 (L_0-4: 1.8562)
    Epoch 1/200, Batch 50/63 - Avg Meta Loss: 1.8438 (L_0-4: 1.8749)
Training MetaModel:   0%|          | 1/200 [16:59<56:20:47, 1019.34s/it]    Epoch 1/200, Final Batch - Avg Meta Loss: 1.8076 (L_0-4: 1.7700)
    Epoch 2/200, Batch 25/63 - Avg Meta Loss: 1.8532 (L_0-4: 1.9306)
    Epoch 2/200, Batch 50/63 - Avg Meta Loss: 1.8256 (L_0-4: 1.8492)
Training MetaModel:   1%|          | 2/200 [1:04:06<114:33:54, 2083.00s/it]    Epoch 2/200, Final Batch - Avg Meta Loss: 1.8135 (L_0-4: 1.7247)
    Epoch 3/200, Batch 25/63 - Avg Meta Loss: 1.8260 (L_0-4: 1.7034)
    Epoch 3/200, Batch 50/63 - Avg Meta Loss: 1.8374 (L_0-4: 1.8373)
Training MetaModel:   2%|▏         | 3/200 [1:21:22<87:48:42, 1604.68s/it]     Epoch 3/200, Final Batch - Avg Meta Loss: 1.8309 (L_0-4: 1.8743)
    Epoch 4/200, Batch 25/63 - Avg Meta Loss: 1.8200 (L_0-4: 1.7904)
    Epoch 4/200, Batch 50/63 - Avg Meta Loss: 1.8323 (L_0-4: 1.7775)
Training MetaModel:   2%|▏         | 4/200 [1:23:17<55:20:24, 1016.45s/it]    Epoch 4/200, Final Batch - Avg Meta Loss: 1.8365 (L_0-4: 1.6249)
    Epoch 5/200, Batch 25/63 - Avg Meta Loss: 1.8167 (L_0-4: 1.9721)
    Epoch 5/200, Batch 50/63 - Avg Meta Loss: 1.8270 (L_0-4: 1.8296)
Training MetaModel:   2%|▎         | 5/200 [1:25:12<37:28:00, 691.70s/it]     Epoch 5/200, Final Batch - Avg Meta Loss: 1.8616 (L_0-4: 1.6108)
    Epoch 6/200, Batch 25/63 - Avg Meta Loss: 1.8340 (L_0-4: 1.7992)
    Epoch 6/200, Batch 50/63 - Avg Meta Loss: 1.8423 (L_0-4: 2.0141)
Training MetaModel:   3%|▎         | 6/200 [1:44:00<45:16:09, 840.05s/it]    Epoch 6/200, Final Batch - Avg Meta Loss: 1.8046 (L_0-4: 1.7554)
    Epoch 7/200, Batch 25/63 - Avg Meta Loss: 1.8364 (L_0-4: 1.9645)
    Epoch 7/200, Batch 50/63 - Avg Meta Loss: 1.8302 (L_0-4: 1.8935)
Training MetaModel:   4%|▎         | 7/200 [1:45:56<32:20:31, 603.27s/it]    Epoch 7/200, Final Batch - Avg Meta Loss: 1.8075 (L_0-4: 1.7013)
    Epoch 8/200, Batch 25/63 - Avg Meta Loss: 1.8306 (L_0-4: 1.7396)
    Epoch 8/200, Batch 50/63 - Avg Meta Loss: 1.8310 (L_0-4: 1.7063)
Training MetaModel:   4%|▍         | 8/200 [2:18:34<55:10:19, 1034.48s/it]    Epoch 8/200, Final Batch - Avg Meta Loss: 1.8423 (L_0-4: 2.0280)
    Epoch 9/200, Batch 25/63 - Avg Meta Loss: 1.8353 (L_0-4: 1.9284)
    Epoch 9/200, Batch 50/63 - Avg Meta Loss: 1.8388 (L_0-4: 1.7543)
Training MetaModel:   4%|▍         | 9/200 [2:51:06<70:06:17, 1321.35s/it]    Epoch 9/200, Final Batch - Avg Meta Loss: 1.8083 (L_0-4: 1.6688)
    Epoch 10/200, Batch 25/63 - Avg Meta Loss: 1.8202 (L_0-4: 1.7861)
    Epoch 10/200, Batch 50/63 - Avg Meta Loss: 1.8595 (L_0-4: 1.8392)
Training MetaModel:   5%|▌         | 10/200 [3:02:32<59:23:17, 1125.25s/it]    Epoch 10/200, Final Batch - Avg Meta Loss: 1.7742 (L_0-4: 1.6124)
    Epoch 11/200, Batch 25/63 - Avg Meta Loss: 1.8508 (L_0-4: 1.7812)
    Epoch 11/200, Batch 50/63 - Avg Meta Loss: 1.8134 (L_0-4: 1.7450)
Training MetaModel:   6%|▌         | 11/200 [3:04:27<42:50:16, 815.96s/it]     Epoch 11/200, Final Batch - Avg Meta Loss: 1.8194 (L_0-4: 1.6707)
    Epoch 12/200, Batch 25/63 - Avg Meta Loss: 1.8328 (L_0-4: 1.6924)
    Epoch 12/200, Batch 50/63 - Avg Meta Loss: 1.8281 (L_0-4: 1.7495)
Training MetaModel:   6%|▌         | 12/200 [3:21:53<46:15:55, 885.93s/it]    Epoch 12/200, Final Batch - Avg Meta Loss: 1.8384 (L_0-4: 1.8846)
    Epoch 13/200, Batch 25/63 - Avg Meta Loss: 1.8037 (L_0-4: 1.9289)
    Epoch 13/200, Batch 50/63 - Avg Meta Loss: 1.8523 (L_0-4: 2.0146)
Training MetaModel:   6%|▋         | 13/200 [4:03:26<71:18:19, 1372.73s/it]    Epoch 13/200, Final Batch - Avg Meta Loss: 1.8401 (L_0-4: 1.7123)
    Epoch 14/200, Batch 25/63 - Avg Meta Loss: 1.8271 (L_0-4: 1.6825)
    Epoch 14/200, Batch 50/63 - Avg Meta Loss: 1.8213 (L_0-4: 1.8506)
Training MetaModel:   7%|▋         | 14/200 [4:27:50<72:20:54, 1400.30s/it]    Epoch 14/200, Final Batch - Avg Meta Loss: 1.8614 (L_0-4: 1.8105)
    Epoch 15/200, Batch 25/63 - Avg Meta Loss: 1.8407 (L_0-4: 1.8642)
    Epoch 15/200, Batch 50/63 - Avg Meta Loss: 1.8482 (L_0-4: 1.8113)
Training MetaModel:   8%|▊         | 15/200 [4:29:44<52:02:10, 1012.60s/it]    Epoch 15/200, Final Batch - Avg Meta Loss: 1.7842 (L_0-4: 1.8107)
    Epoch 16/200, Batch 25/63 - Avg Meta Loss: 1.8354 (L_0-4: 1.8120)
    Epoch 16/200, Batch 50/63 - Avg Meta Loss: 1.8310 (L_0-4: 1.8228)
Training MetaModel:   8%|▊         | 16/200 [4:48:34<53:33:36, 1047.92s/it]    Epoch 16/200, Final Batch - Avg Meta Loss: 1.8360 (L_0-4: 2.0460)
    Epoch 17/200, Batch 25/63 - Avg Meta Loss: 1.8371 (L_0-4: 1.9928)
    Epoch 17/200, Batch 50/63 - Avg Meta Loss: 1.8407 (L_0-4: 1.9415)
Training MetaModel:   8%|▊         | 17/200 [4:50:22<38:54:40, 765.47s/it]     Epoch 17/200, Final Batch - Avg Meta Loss: 1.8161 (L_0-4: 1.8338)
    Epoch 18/200, Batch 25/63 - Avg Meta Loss: 1.8487 (L_0-4: 1.8544)
    Epoch 18/200, Batch 50/63 - Avg Meta Loss: 1.8138 (L_0-4: 1.7967)
Training MetaModel:   9%|▉         | 18/200 [5:04:38<40:04:17, 792.62s/it]    Epoch 18/200, Final Batch - Avg Meta Loss: 1.8428 (L_0-4: 1.8466)
    Epoch 19/200, Batch 25/63 - Avg Meta Loss: 1.8498 (L_0-4: 1.7444)
    Epoch 19/200, Batch 50/63 - Avg Meta Loss: 1.8190 (L_0-4: 1.9204)
Training MetaModel:  10%|▉         | 19/200 [5:34:33<54:58:33, 1093.44s/it]    Epoch 19/200, Final Batch - Avg Meta Loss: 1.8202 (L_0-4: 1.8220)
    Epoch 20/200, Batch 25/63 - Avg Meta Loss: 1.8352 (L_0-4: 1.8357)
    Epoch 20/200, Batch 50/63 - Avg Meta Loss: 1.8329 (L_0-4: 1.7854)
Training MetaModel:  10%|█         | 20/200 [5:49:14<51:29:50, 1029.95s/it]    Epoch 20/200, Final Batch - Avg Meta Loss: 1.8328 (L_0-4: 1.7007)
    Epoch 21/200, Batch 25/63 - Avg Meta Loss: 1.8158 (L_0-4: 1.8976)
    Epoch 21/200, Batch 50/63 - Avg Meta Loss: 1.8366 (L_0-4: 1.7796)
Training MetaModel:  10%|█         | 21/200 [5:51:02<37:26:14, 752.93s/it]     Epoch 21/200, Final Batch - Avg Meta Loss: 1.8483 (L_0-4: 1.8057)
    Epoch 22/200, Batch 25/63 - Avg Meta Loss: 1.8165 (L_0-4: 1.7626)
    Epoch 22/200, Batch 50/63 - Avg Meta Loss: 1.8531 (L_0-4: 2.0057)
Training MetaModel:  11%|█         | 22/200 [5:52:50<27:40:00, 559.55s/it]    Epoch 22/200, Final Batch - Avg Meta Loss: 1.8377 (L_0-4: 1.9751)
    Epoch 23/200, Batch 25/63 - Avg Meta Loss: 1.8275 (L_0-4: 1.8515)
    Epoch 23/200, Batch 50/63 - Avg Meta Loss: 1.8382 (L_0-4: 1.8422)
Training MetaModel:  12%|█▏        | 23/200 [5:54:40<20:52:13, 424.48s/it]    Epoch 23/200, Final Batch - Avg Meta Loss: 1.8412 (L_0-4: 1.8374)
    Epoch 24/200, Batch 25/63 - Avg Meta Loss: 1.8307 (L_0-4: 1.8362)
    Epoch 24/200, Batch 50/63 - Avg Meta Loss: 1.8243 (L_0-4: 1.7848)
Training MetaModel:  12%|█▏        | 24/200 [5:56:30<16:08:27, 330.16s/it]    Epoch 24/200, Final Batch - Avg Meta Loss: 1.8717 (L_0-4: 2.0374)
    Epoch 25/200, Batch 25/63 - Avg Meta Loss: 1.8367 (L_0-4: 1.8124)
    Epoch 25/200, Batch 50/63 - Avg Meta Loss: 1.8357 (L_0-4: 1.7775)
Training MetaModel:  12%|█▎        | 25/200 [5:58:20<12:50:53, 264.30s/it]    Epoch 25/200, Final Batch - Avg Meta Loss: 1.8051 (L_0-4: 1.5530)
    Epoch 26/200, Batch 25/63 - Avg Meta Loss: 1.8320 (L_0-4: 1.8308)
    Epoch 26/200, Batch 50/63 - Avg Meta Loss: 1.8187 (L_0-4: 1.8345)
Training MetaModel:  13%|█▎        | 26/200 [6:00:11<10:33:08, 218.33s/it]    Epoch 26/200, Final Batch - Avg Meta Loss: 1.8516 (L_0-4: 1.8590)
    Epoch 27/200, Batch 25/63 - Avg Meta Loss: 1.8382 (L_0-4: 1.7721)
    Epoch 27/200, Batch 50/63 - Avg Meta Loss: 1.8198 (L_0-4: 1.7705)
Training MetaModel:  14%|█▎        | 27/200 [6:03:21<10:04:13, 209.56s/it]    Epoch 27/200, Final Batch - Avg Meta Loss: 1.8376 (L_0-4: 1.7083)
    Epoch 28/200, Batch 25/63 - Avg Meta Loss: 1.8291 (L_0-4: 1.6928)
    Epoch 28/200, Batch 50/63 - Avg Meta Loss: 1.8240 (L_0-4: 1.8016)
Training MetaModel:  14%|█▍        | 28/200 [6:09:30<12:18:12, 257.51s/it]    Epoch 28/200, Final Batch - Avg Meta Loss: 1.8394 (L_0-4: 1.6676)
    Epoch 29/200, Batch 25/63 - Avg Meta Loss: 1.8256 (L_0-4: 2.0726)
    Epoch 29/200, Batch 50/63 - Avg Meta Loss: 1.8382 (L_0-4: 2.0114)
Training MetaModel:  14%|█▍        | 29/200 [6:12:32<11:09:13, 234.82s/it]    Epoch 29/200, Final Batch - Avg Meta Loss: 1.8475 (L_0-4: 1.9027)
    Epoch 30/200, Batch 25/63 - Avg Meta Loss: 1.8007 (L_0-4: 1.6686)
    Epoch 30/200, Batch 50/63 - Avg Meta Loss: 1.8544 (L_0-4: 1.8956)
Training MetaModel:  15%|█▌        | 30/200 [6:15:09<9:59:28, 211.58s/it]     Epoch 30/200, Final Batch - Avg Meta Loss: 1.8455 (L_0-4: 1.8723)
    Epoch 31/200, Batch 25/63 - Avg Meta Loss: 1.8267 (L_0-4: 1.8197)
    Epoch 31/200, Batch 50/63 - Avg Meta Loss: 1.8353 (L_0-4: 1.9416)
Training MetaModel:  16%|█▌        | 31/200 [6:17:55<9:17:16, 197.85s/it]    Epoch 31/200, Final Batch - Avg Meta Loss: 1.8257 (L_0-4: 1.5058)
    Epoch 32/200, Batch 25/63 - Avg Meta Loss: 1.8201 (L_0-4: 1.8200)
    Epoch 32/200, Batch 50/63 - Avg Meta Loss: 1.8491 (L_0-4: 1.7909)
Training MetaModel:  16%|█▌        | 32/200 [6:20:48<8:53:32, 190.55s/it]    Epoch 32/200, Final Batch - Avg Meta Loss: 1.8207 (L_0-4: 1.7743)
    Epoch 33/200, Batch 25/63 - Avg Meta Loss: 1.8445 (L_0-4: 1.9730)
    Epoch 33/200, Batch 50/63 - Avg Meta Loss: 1.8246 (L_0-4: 1.7790)
Training MetaModel:  16%|█▋        | 33/200 [6:23:41<8:35:17, 185.13s/it]    Epoch 33/200, Final Batch - Avg Meta Loss: 1.8220 (L_0-4: 1.7811)
    Epoch 34/200, Batch 25/63 - Avg Meta Loss: 1.8315 (L_0-4: 1.7666)
    Epoch 34/200, Batch 50/63 - Avg Meta Loss: 1.8532 (L_0-4: 1.8278)
Training MetaModel:  17%|█▋        | 34/200 [6:26:30<8:18:26, 180.16s/it]    Epoch 34/200, Final Batch - Avg Meta Loss: 1.7918 (L_0-4: 1.8164)
    Epoch 35/200, Batch 25/63 - Avg Meta Loss: 1.8304 (L_0-4: 1.7158)
    Epoch 35/200, Batch 50/63 - Avg Meta Loss: 1.8311 (L_0-4: 1.7756)
Training MetaModel:  18%|█▊        | 35/200 [7:36:25<63:28:15, 1384.82s/it]    Epoch 35/200, Final Batch - Avg Meta Loss: 1.8268 (L_0-4: 1.6410)
    Epoch 36/200, Batch 25/63 - Avg Meta Loss: 1.8572 (L_0-4: 2.0331)
    Epoch 36/200, Batch 50/63 - Avg Meta Loss: 1.8030 (L_0-4: 1.9363)
Training MetaModel:  18%|█▊        | 36/200 [7:38:12<45:37:14, 1001.43s/it]    Epoch 36/200, Final Batch - Avg Meta Loss: 1.8343 (L_0-4: 1.6064)
    Epoch 37/200, Batch 25/63 - Avg Meta Loss: 1.8226 (L_0-4: 1.8192)
    Epoch 37/200, Batch 50/63 - Avg Meta Loss: 1.8486 (L_0-4: 1.8496)
Training MetaModel:  18%|█▊        | 37/200 [7:56:35<46:43:15, 1031.88s/it]    Epoch 37/200, Final Batch - Avg Meta Loss: 1.8338 (L_0-4: 1.8510)
    Epoch 38/200, Batch 25/63 - Avg Meta Loss: 1.8444 (L_0-4: 1.7722)
Training MetaModel:  18%|█▊        | 37/200 [8:01:56<35:23:07, 781.52s/it]